In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pathlib
import keras
import tensorflow as tf
from tensorflow.keras import layers
import cv2

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import random

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir("/content/gdrive/MyDrive/ColabNotebooks/ClassificationCarsML")

In [ ]:
data_path = "/content/gdrive/MyDrive/ColabNotebooks/ClassificationCarsML/sharePublic/Public"
dataset_path = "/content/gdrive/MyDrive/ColabNotebooks/ClassificationCarsML/Public"
col_name = ['ImageFullPath', 'CategoryID']
AUTOTUNE = tf.data.AUTOTUNE
img_height = 128
img_width = 128
car_brands = ['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast']
categories = {
    'Others': 0,
    'Honda': 1,
    'Hyundai': 2,
    'KIA': 3,
    'Mazda': 4,
    'Mitsubishi': 5,
    'Suzuki': 6,
    'Toyota': 7,
    'VinFast': 8,
}

# Preprocess dataset

---



In [ ]:
num_ds = 1
train_df = pd.read_csv(dataset_path + f'/CarDataset-Splits-{num_ds}-Train.csv')
test_df = pd.read_csv(dataset_path + f'/CarDataset-Splits-{num_ds}-Test.csv')
train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_df[col_name[0]].values, train_df[col_name[1]].values))
test_dataset = tf.data.Dataset.from_tensor_slices((test_df[col_name[0]].values, test_df[col_name[1]].values))

In [ ]:
def read_image(image_path):
    img = tf.io.read_file(data_path + '/'+ image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.squeeze(img)
    img.set_shape([None, None, 3])
    img = tf.image.central_crop(img, random.uniform(0.7, 1.0))
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, random.uniform(0.1, 0.2))
    img = tf.image.random_contrast(img, 0.8, 1.2)
    #A highly saturated image appears vibrant and colorful, while a low saturation image looks more subdued or muted
    img = tf.image.random_saturation(img, 0.8, 1.2)
    img = tf.image.random_hue(img, random.uniform(0.1, 0.2))
    img = tf.clip_by_value(img, 0, 255)
    return img

In [ ]:
# num_parallel_calls=tf.data.AUTOTUNE: optimize the data processing performance
train_ds = train_dataset.map(lambda path, category_id: (read_image(path), tf.cast(category_id, tf.int32)))
test_ds = test_dataset.map(lambda path, category_id: (read_image(path), tf.cast(category_id, tf.int32)))

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
train_ds = train_ds.map(lambda path, category_id: (normalization_layer(path), category_id), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(lambda path, category_id: (normalization_layer(path), category_id), num_parallel_calls=AUTOTUNE)

In [ ]:
# No need to run this code, the code above has shuffle already
# train_ds = train_ds.shuffle(len(train_ds))

In [ ]:
batch_size = 32
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [ ]:
#Khong chay cai nay

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(car_brands[int(labels[i])])
    plt.axis("off")

In [ ]:
#Khong chay cai nay
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 128, 128, 3)
(32,)


In [ ]:
#Khong chay cai nay
for image_batch, labels_batch in train_ds:
  print(image_batch.numpy())
  print(labels_batch.numpy())
  break

[[[[0.40874696 0.5686275  0.7019608 ]
   [0.40874696 0.5686275  0.7019608 ]
   [0.4156863  0.57254905 0.7058824 ]
   ...
   [0.43307796 0.60170543 0.7311172 ]
   [0.44796264 0.6009038  0.7303156 ]
   [0.44176987 0.59471107 0.7241228 ]]

  [[0.40214464 0.5707721  0.70018387]
   [0.4039216  0.57254905 0.7019608 ]
   [0.41176474 0.5686275  0.7019608 ]
   ...
   [0.42895222 0.6        0.7282016 ]
   [0.43310955 0.5990256  0.7294118 ]
   [0.427451   0.59336704 0.7254902 ]]

  [[0.40000004 0.5686275  0.69803923]
   [0.40000004 0.5686275  0.69803923]
   [0.41176474 0.5686275  0.7019608 ]
   ...
   [0.42352945 0.5966759  0.72881436]
   [0.40963545 0.5927543  0.71824455]
   [0.4057139  0.58883274 0.714323  ]]

  ...

  [[0.41788188 0.41003874 0.42964658]
   [0.41743264 0.4095895  0.42919734]
   [0.39968815 0.39184502 0.41145286]
   ...
   [0.10588236 0.12156864 0.13333334]
   [0.10196079 0.11764707 0.12941177]
   [0.10196079 0.11764707 0.12941177]]

  [[0.35686275 0.34901962 0.36862746]
   [0.3

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
save_path = "/content/gdrive/MyDrive/ColabNotebooks/ClassificationCarsML/SaveModel"

In [ ]:
model = tf.keras.models.load_model(os.path.join(save_path, "my_model.keras"))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ random_rotation (RandomRotation)     │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 63, 63, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 30, 30, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 96)          │          55,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 14, 14, 96)          │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 96)          │          83,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 96)            │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 6, 6, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 4, 4, 64)            │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 2, 2, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 2, 2, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 2, 2, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │              

 Total params: 941,917 (3.59 MB)

 Trainable params: 313,737 (1.20 MB)

 Non-trainable params: 704 (2.75 KB)

 Optimizer params: 627,476 (2.39 MB)

In [ ]:
total_df = pd.read_csv(dataset_path + f'/CarDataset.csv')

In [ ]:
predict_img = total_df[total_df['CategoryID'] == categories['Hyundai']].sample(1)
predict_img

,ImageFullPath,CategoryID
3047,Hyundai/22521070-22520211.Hyundai.129.jpg,2


In [ ]:
predict_img = total_df.sample(1)
predict_img

,ImageFullPath,CategoryID
18771,Suzuki/21522500-21522771.Suzuki.94.jpg,6


In [ ]:
predict_img_path = predict_img['ImageFullPath'].values[0]
predict_img_true_label = predict_img['CategoryID'].values[0]
predict_img_numpy = read_image(predict_img_path)
predict_img_numpy = normalization_layer(predict_img_numpy)

In [ ]:
predictions  = model.predict(tf.expand_dims(predict_img_numpy, 0), verbose=0)
predicted_label = car_brands[np.argmax(predictions[0])]
print("Predicted label:", predicted_label)
print("True label:", car_brands[predict_img_true_label])

Predicted label: Hyundai
True label: Hyundai


In [ ]:
predictions

array([[-29.781612  ,  -4.9491134 ,   1.2609446 ,  -0.03099945,
         -5.527566  ,  -6.2264013 ,  -2.9122925 ,   0.9686952 ,
         -3.8974974 ]], dtype=float32)

In [ ]:
epochs=15
model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=epochs
)

Epoch 1/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 370s 626ms/step - accuracy: 0.5173 - loss: 1.3734 - val_accuracy: 0.6219 - val_loss: 1.0939
Epoch 2/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.5736 - loss: 1.2276 - val_accuracy: 0.6102 - val_loss: 1.1181
Epoch 3/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.5872 - loss: 1.1811 - val_accuracy: 0.6137 - val_loss: 1.1223
Epoch 4/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.5943 - loss: 1.1570 - val_accuracy: 0.6244 - val_loss: 1.0898
Epoch 5/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.6044 - loss: 1.1271 - val_accuracy: 0.6085 - val_loss: 1.1441
Epoch 6/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.6120 - loss: 1.1094 - val_accuracy: 0.6121 - val_loss: 1.1160
Epoch 7/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.6159 - loss: 1.0961 - val_accuracy: 0.6130 - val_loss: 1.1164
Epoch 8/15
591/591 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.6095 - loss: 1.1010 - val_a

In [ ]:
model.save(os.path.join(save_path, "my_model.keras"))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, RandomZoom, RandomRotation, Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Input(shape=(img_height, img_width, 3)))
model.add(layers.RandomRotation(0.1))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())


model.add(layers.Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(len(car_brands)))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])